In [1]:
%matplotlib widget
import os
import glob
import h5py
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from scipy.signal import find_peaks
from scipy.optimize import curve_fit

In [2]:
def average_waveform(waveforms, min_height=500):
    peak_array = []
    average_waveform = np.array([0]*len(waveforms[0]))
    N = 0
    for waveform in tqdm(waveforms, total=waveforms.shape[0]):
        peak_locs = find_peaks(waveform, height=min_height, distance=5)[0]
        if len(peak_locs) == 0: continue
        if len(peak_locs) >= 2:
            if peak_locs[1] < 1000: continue
        if (waveform[peak_locs[0]] < 1000) & (peak_locs[0] < 500):
            average_waveform = average_waveform + waveform
            N += 1
    return average_waveform/N

In [3]:
def exp_func(t, A, tau, D):
    return A*np.exp(-t/tau) + D

def double_exp(t, A1, A2, tau1, tau2, D):
    return exp_func(t, A1, tau1, 0) + exp_func(t, A2, tau2, D)

def rise_func(t, A, t0, tau, D):
    return A*(1 - np.exp(-(t-t0)/tau)) + D

In [4]:
data_path = "/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1"
file_list = glob.glob(data_path+"/*.h5")

In [5]:
file_list

['/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_25.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_26.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_27.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_28.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_29.h5',
 '/Volumes/TOSHIBA_EXT/sipm_data_2/ketek_box/ketek_room2/t1/t1_ketek_room2_30.h5']

In [6]:
sipm_file = h5py.File(file_list[3], "r")
waveforms = sipm_file["/raw/waveforms"][:]
baselines = sipm_file["/raw/baselines"][:]
base_forms = waveforms - baselines
sipm_file.close()

In [7]:
plt.figure()
plt.plot(base_forms[3])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
super_pulse = average_waveform(base_forms)
t = np.arange(0, 2*len(super_pulse), 2)

In [13]:
plt.figure()
plt.plot(t, super_pulse)
plt.plot(t[58:400], super_pulse[58:400])
plt.xlim(0, 1000)
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (ADC)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Amplitude (ADC)')

In [14]:
left_bound = 58
right_bound = 400
t_fit = t[left_bound:right_bound]
pulse_fit = super_pulse[left_bound:right_bound]

In [18]:
coeffs, covs = curve_fit(exp_func, t_fit, pulse_fit, p0=[100, 50, 0])
errors = np.sqrt(np.diag(covs))

In [19]:
print(coeffs)
print(errors)

[7154.62095796   53.57366875   41.5179915 ]
[63.69142988  0.183377    0.30830826]


In [21]:
plt.figure()
plt.plot(t, super_pulse)
plt.plot(t_fit, exp_func(t_fit, *coeffs))
plt.xlim(0, 1000)
plt.xlabel("Time (ns)")
plt.ylabel("Amplitude (ADC)")

#print("tau1: ", coeffs[2], "tau2: ", coeffs[3])
print("tau1: ", coeffs[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

tau1:  53.573668750252
